In [42]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [62]:
!wget -q -O 'toronto_data.csv' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


In [64]:
toronto_data = ['features']

In [66]:
toronto_data[0]

'features'

In [67]:
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

neighborhoods = pd.DataFrame(columns=column_names)

In [68]:
neighborhoods

,PostalCode,Borough,Neighborhood


In [74]:
for data in toronto_data:
    borough = neighborhood_name = data['borough'] 
    neighborhood_name = data['name']
    Postalcode = data['Postalcode']
        
   
    neighborhoods = neighborhoods.append({'PostalCode': postal_code,
                                          'Borough': borough,
                                          'Neighborhood': neighborhood_name}
                                          , ignore_index=True)

TypeError: string indices must be integers